<a href="https://colab.research.google.com/github/Sruthij93/Codebase-RAG/blob/main/SJ_Codebase_RAG_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![Img](https://app.theheadstarter.com/static/hs-logo-opengraph.png)

# Headstarter Codebase RAG Project

![Screenshot 2024-11-25 at 7 12 58 PM](https://github.com/user-attachments/assets/0bd67cf0-43d5-46d2-879c-a752cae4c8e3)

# Install Necessary Libraries

In [1]:
! pip install pygithub langchain langchain-community openai tiktoken pinecone-client langchain_pinecone sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.9/375.9 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.5/409.5 kB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.1 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successf

In [2]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from langchain_pinecone import PineconeVectorStore
from langchain.embeddings import OpenAIEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
from google.colab import userdata
from pinecone import Pinecone
import os
import tempfile
from github import Github, Repository
from git import Repo
from openai import OpenAI
from pathlib import Path
from langchain.schema import Document
from pinecone import Pinecone

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


# Clone a GitHub Repo locally

In [3]:
def clone_repo(repo_url):
  repo_name = repo_url.split("/")[-1]
  repo_path = f"/content/{repo_name}"
  Repo.clone_from(repo_url, str(repo_path))
  return str(repo_name)

In [4]:
path = clone_repo("https://github.com/CoderAgent/SecureAgent")

In [14]:
path = "/content/SecureAgent"

In [20]:
SUPPORTED_EXTENSIONS = [".py", ".js", ".tsx", ".ts", ".java",".cpp"]

#directories to be ignored. do not commit node_modules, .git for eg
IGNORED_DIRS = [".git", "node_modules", "dist", "__pycache__", ".next", ".vscode", ".env", "venv"]

In [21]:
def get_file_content(file_path, repo_path):
  #error handling
  try:

    with open(file_path, "r", encoding = "utf-8") as f:
      content = f.read()

      rel_path = os.path.relpath(file_path, repo_path)

      return {
          "name": rel_path,
          "content": content
      }

  except Exception as e:
    print(f"Error reading file {file_path} : {e}")
    return None


In [22]:
def get_main_files_content(repo_path: str):
   """
   Get content of supported code files from the local repository.


   Args:
       repo_path: Path to the local repository


   Returns:
       List of dictionaries containing file names and contents
   """
   files_content = []


   try:
       for root, _, files in os.walk(repo_path):
           # Skip if current directory is in ignored directories
           if any(ignored_dir in root for ignored_dir in IGNORED_DIRS):
               continue


           # Process each file in current directory
           for file in files:
               file_path = os.path.join(root, file)
               if os.path.splitext(file)[1] in SUPPORTED_EXTENSIONS:
                   file_content = get_file_content(file_path, repo_path)
                   if file_content:
                       files_content.append(file_content)


   except Exception as e:
       print(f"Error reading repository: {str(e)}")


   return files_content

In [23]:
file_content = get_main_files_content(path)

In [24]:
file_content

[{'name': 'src/constants.ts',
  'content': 'import { Node } from "@babel/traverse";\nimport { JavascriptParser } from "./context/language/javascript-parser";\nimport { ChatCompletionMessageParam } from "groq-sdk/resources/chat/completions";\n\nexport interface PRFile {\n  sha: string;\n  filename: string;\n  status:\n    | "added"\n    | "removed"\n    | "renamed"\n    | "changed"\n    | "modified"\n    | "copied"\n    | "unchanged";\n  additions: number;\n  deletions: number;\n  changes: number;\n  blob_url: string;\n  raw_url: string;\n  contents_url: string;\n  patch?: string;\n  previous_filename?: string;\n  patchTokenLength?: number;\n  old_contents?: string;\n  current_contents?: string;\n}\n\nexport interface BuilderResponse {\n  comment: string;\n  structuredComments: any[];\n}\n\nexport interface Builders {\n  convoBuilder: (diff: string) => ChatCompletionMessageParam[];\n  responseBuilder: (feedbacks: string[]) => Promise<BuilderResponse>;\n}\n\nexport interface PatchInfo {\

# Embeddings

In [25]:
def get_huggingface_embeddings(text, model_name="sentence-transformers/all-mpnet-base-v2"):
    model = SentenceTransformer(model_name)
    return model.encode(text)

In [26]:
text = "This is a sample sentence"
embeddings = get_huggingface_embeddings(text)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [27]:
embeddings

array([ 3.38363163e-02, -8.39076266e-02, -3.01864091e-02, -1.50361750e-02,
       -8.36341828e-02,  2.67099831e-02, -9.17343516e-03,  3.30600888e-03,
       -4.73221056e-02, -1.27414269e-02,  8.20042789e-02,  2.05611922e-02,
        1.98337398e-02, -3.27859670e-02,  1.76552087e-02, -6.64511994e-02,
        5.29538617e-02,  2.12427732e-02, -4.35398743e-02,  2.42592040e-02,
        3.84509680e-03,  1.64461471e-02,  3.46780345e-02,  2.50417367e-02,
        2.40206383e-02, -3.28408033e-02,  5.68543281e-03, -2.87025869e-02,
       -6.85418257e-03, -3.71621139e-02,  1.50979077e-02, -1.02682291e-02,
       -3.70136090e-03, -1.02162957e-01,  1.71727061e-06, -3.04962508e-03,
       -8.88470747e-03, -1.37526616e-02, -5.74572645e-02,  3.62290107e-02,
        4.53157583e-03,  6.52435124e-02, -1.65319722e-03,  5.79803586e-02,
       -1.97080094e-02,  2.01019887e-02,  5.05360886e-02,  3.05137169e-02,
       -3.38396840e-02,  5.72957583e-02, -4.07251203e-03, -1.28480038e-02,
       -4.37915474e-02, -

# Setting up Pinecone
**1. Create an account on [Pinecone.io](https://app.pinecone.io/)**

**2. Create a new index called "codebase-rag" and set the dimensions to 768. Leave the rest of the settings as they are.**

![Screenshot 2024-11-24 at 10 58 50 PM](https://github.com/user-attachments/assets/f5fda046-4087-432a-a8c2-86e061005238)



**3. Create an API Key for Pinecone**

![Screenshot 2024-11-24 at 10 44 37 PM](https://github.com/user-attachments/assets/e7feacc6-2bd1-472a-82e5-659f65624a88)


**4. Store your Pinecone API Key within Google Colab's secrets section, and then enable access to it (see the blue checkmark)**

![Screenshot 2024-11-24 at 10 45 25 PM](https://github.com/user-attachments/assets/eaf73083-0b5f-4d17-9e0c-eab84f91b0bc)



In [28]:
# Set the PINECONE_API_KEY as an environment variable
pinecone_api_key = userdata.get("PINECONE_API_KEY")
os.environ['PINECONE_API_KEY'] = pinecone_api_key

# Initialize Pinecone
pc = Pinecone(api_key=userdata.get("PINECONE_API_KEY"),)

# Connect to your Pinecone index
pinecone_index = pc.Index("codebase-rag")

In [29]:
vectorstore = PineconeVectorStore(index_name="codebase-rag", embedding=HuggingFaceEmbeddings())

<ipython-input-29-5982ffb8f713>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  vectorstore = PineconeVectorStore(index_name="codebase-rag", embedding=HuggingFaceEmbeddings())
<ipython-input-29-5982ffb8f713>:1: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  vectorstore = PineconeVectorStore(index_name="codebase-rag", embedding=HuggingFaceEmbeddings())


In [31]:
# Insert the codebase embeddings into Pinecone
documents = []

for file in file_content:
  doc = Document(
      page_content = f"{file['name']}\n{file['content']}",
      metadata = {"source" : file['name']} # keep track of the path of the file if it is a large codespace
  )
  documents.append(doc)

vectorstore = PineconeVectorStore.from_documents(
    documents=documents,
    embedding=HuggingFaceEmbeddings(),
    index_name="codebase-rag",
    namespace="https://github.com/CoderAgent/SecureAgent"
) # insert into pinecone directly









<ipython-input-31-ce51c68d54c3>:13: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embedding=HuggingFaceEmbeddings(),


# Perform RAG

1. Get your Groq API Key [here](https://console.groq.com/keys)

2. Paste your Groq API Key into your Google Colab secrets, and make sure to enable permissions for it

![Screenshot 2024-11-25 at 12 00 16 AM](https://github.com/user-attachments/assets/e5525d29-bca6-4dbd-892b-cc770a6b281d)


In [32]:
client = OpenAI(
    base_url="https://api.groq.com/openai/v1",
    api_key=userdata.get("GROQ_API_KEY")
)

In [33]:
query = "how can we add a python parser to the code?"

In [34]:
query_embedding = get_huggingface_embeddings(query)

In [35]:
query_embedding

array([ 3.50039601e-02,  3.19803283e-02, -7.54642487e-03,  2.32072212e-02,
        3.33503373e-02,  4.88003753e-02, -2.38095839e-02, -5.38620446e-03,
        1.50125148e-02, -7.15338364e-02, -2.86776363e-03,  6.33163154e-02,
       -2.03450434e-02,  8.41649249e-02, -1.49502223e-02, -4.50661071e-02,
        2.00629346e-02,  9.03427415e-03,  8.35644230e-02,  3.27417180e-02,
       -3.54465097e-02,  3.37883793e-02,  3.68302651e-02,  3.07752956e-02,
        3.72014269e-02,  1.62651502e-02, -2.53158044e-02, -3.03759519e-02,
       -1.11641465e-02, -8.61920416e-02, -3.09500061e-02, -7.24871457e-03,
        6.48079589e-02,  1.04105528e-02,  1.29354055e-06, -1.60781015e-02,
       -4.96336631e-02,  1.59990776e-03, -2.66894829e-02, -1.85946599e-02,
        5.54179214e-02, -2.98578404e-02,  3.04939318e-02, -2.17241216e-02,
       -8.02262127e-02, -1.65030323e-02,  2.15911642e-02, -5.43753803e-02,
        5.79174869e-02,  7.15116858e-02,  1.52468830e-02, -2.46721562e-02,
        4.46812715e-03,  

In [36]:
top_matches = pinecone_index.query(
    vector=query_embedding.tolist(),
    top_k=5,
    include_metadata=True,
    namespace="https://github.com/CoderAgent/SecureAgent"
)

In [37]:
top_matches

{'matches': [{'id': '6c741f5c-2614-44ae-af39-6d8d50e00782',
              'metadata': {'source': 'src/context/language/python-parser.ts',
                           'text': 'src/context/language/python-parser.ts\n'
                                   'import { AbstractParser, EnclosingContext '
                                   '} from "../../constants";\n'
                                   'export class PythonParser implements '
                                   'AbstractParser {\n'
                                   '  findEnclosingContext(\n'
                                   '    file: string,\n'
                                   '    lineStart: number,\n'
                                   '    lineEnd: number\n'
                                   '  ): EnclosingContext {\n'
                                   '    // TODO: Implement this method for '
                                   'Python\n'
                                   '    return null;\n'
                          

In [40]:
contexts = [item["metadata"]["text"] for item in top_matches["matches"]]

In [41]:
augmented_query = "<CONTEXT>\n" + "\n\n-------\n\n".join(contexts[:10]) + "\n-------\n</CONTEXT>\n\n\n\nMY QUESTION:\n" + query

In [45]:
print(augmented_query)

<CONTEXT>
src/context/language/python-parser.ts
import { AbstractParser, EnclosingContext } from "../../constants";
export class PythonParser implements AbstractParser {
  findEnclosingContext(
    file: string,
    lineStart: number,
    lineEnd: number
  ): EnclosingContext {
    // TODO: Implement this method for Python
    return null;
  }
  dryRun(file: string): { valid: boolean; error: string } {
    // TODO: Implement this method for Python
    return { valid: false, error: "Not implemented yet" };
  }
}


-------

src/prompts.ts
import { encode, encodeChat } from "gpt-tokenizer";
import type { ChatCompletionMessageParam } from "groq-sdk/resources/chat/completions";
import type { PRFile } from "./constants";
import {
  rawPatchStrategy,
  smarterContextPatchStrategy,
} from "./context/review";
import { GROQ_MODEL, type GroqChatModel } from "./llms/groq";

const ModelsToTokenLimits: Record<GroqChatModel, number> = {
  "mixtral-8x7b-32768": 32768,
  "gemma-7b-it": 32768,
  "llama3

In [52]:
system_prompt = f"""you are a senior software engineer, specializing in typescript.

Answer any questions I have about the codebase, based on all the context provided.
Always consider all of the context provided when forming the response.

Let's think step by step."""


try:
      llm_response = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": augmented_query}
        ]
    )

      response = llm_response.choices[0].message.content
catch:
      llm_response = client.chat.completions.create(
        model="llama-3.1-70b-versatile",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": augmented_query}
        ]
    )

      response = llm_response.choices[0].message.content


In [51]:
print(response)

You can add a Python parser to the code by creating a new class that implements the `AbstractParser` interface, and uses a Python parser library such as `pygments` or `asteval` to analyze the Python code.

Here is an example of how you can do it:

```typescript
// src/context/language/python-parser.ts
import { AbstractParser, EnclosingContext } from "../../constants";

import * as pythonparser from "python-parser";
import * as spyderlib from "spyder-ide-python-parser";

const parsePythonFile = (file: string) => {
  try {
    const pythonAST = pythonparser.parse(file);
    // You can use the AST to find the enclosing context
    return {
      enclosingContext: pythonAST,
    } as EnclosingContext;
  } catch (e) {
    // For now, we will just return an empty object
    return {
      enclosingContext: null,
    } as EnclosingContext;
  }
};

const dryRunPythonFile = (file: string) => {
  try {
    return { valid: true, error: "" };
  } catch (e) {
    return { valid: false, error: e };


In [53]:
def perform_rag(query):
   raw_query_embedding = get_huggingface_embeddings(query)


   top_matches = pinecone_index.query(vector=raw_query_embedding.tolist(), top_k=5, include_metadata=True, namespace="https://github.com/CoderAgent/SecureAgent")


   # Get the list of retrieved texts
   contexts = [item['metadata']['text'] for item in top_matches['matches']]


   augmented_query = "<CONTEXT>\n" + "\n\n-------\n\n".join(contexts[ : 10]) + "\n-------\n</CONTEXT>\n\n\n\nMY QUESTION:\n" + query


   # Modify the prompt below as need to improve the response quality
   system_prompt = f"""You are a Senior Software Engineer, specializing in TypeScript.


   Answer any questions I have about the codebase, based on the code provided. Always consider all of the context provided when forming a response.
   """


   llm_response = client.chat.completions.create(
       model="llama-3.1-8b-instant",
       messages=[
           {"role": "system", "content": system_prompt},
           {"role": "user", "content": augmented_query}
       ]
   )


   return llm_response.choices[0].message.content

In [55]:
response = perform_rag("how can i improve the quality of feedback from the LLM?")

print(response)

Improving the quality of feedback from the LLM involves several strategies. I'll outline a few suggestions based on the provided code and context.

1. **Fine-tune the LLM model**: The current implementation uses a general-purpose LLM model (GROQ_MODEL) for both the review diffs and XML review prompts. Consider fine-tuning a more specialized model for review tasks, such as a model specifically trained for code review, to improve the quality of feedback.

2. **Use a more accurate diff prompt**: The current `REVIEW_DIFF_PROMPT` and `XML_PR_REVIEW_PROMPT` may not be specific or precise enough to guide the LLM's output. Consider using more technical terms, code-specific language, or incorporating more context about the codebase to help the model provide more accurate feedback.

3. **Add more specific feedback prompts**: The `INLINE_FIX_PROMPT` uses a general prompt to guide the LLM to provide code fixes. Consider adding more specific prompts for different types of feedback, such as suggesti